In [6]:
#Boilerplate imports
import pandas as pd
import numpy as np
import random
import scipy
import itertools
from collections import Counter
from collections import defaultdict
import functools
import math
from sklearn.model_selection import train_test_split

In [7]:
##Import data

#file paths
#low expression population
raws1path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S1.fa.uniq.tcga.all.csv'
raws3path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S3.fa.uniq.tcga.all.csv'
raws5path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S5.fa.uniq.tcga.all.csv'
#high expression population
raws2path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S2.fa.uniq.tcga.all.csv'
raws4path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S4.fa.uniq.tcga.all.csv'
raws6path = 'D:/Projects/iSynPro/SynPro/Complete Raw data for sorted cells/S6.fa.uniq.tcga.all.csv'

#note order of list; low expressors then high espressor datasets
path_list = [raws1path, raws3path, raws5path, raws2path, raws4path, raws6path]

#read in a csv with a single column, split to divide '# of hits' from nucleotide sequence, filter out sequences with < 5 hits
def read_in(filepath):
    dfs1 = pd.read_csv(filepath, engine='python', sep=None, header=None, names=['count','sequence'])
    dfs2 = dfs1.loc[dfs1['count'] > 9]
    return dfs2

#clean up read in sequences, remove sequencing artifact
def clean_read(raws1path):
    p1_raw_df = read_in(raws1path)
    #p1_raw_df['sequence'].replace(regex=True,inplace=True,to_replace=r'TCGAGTAGAGTCTAGACTCTACATTTTGACACCCCCA',value=r'')
    il2mpseq = 'ACATTTTGACACCCCCATAATATTTTTCCAGAATTAACAGTATAAATTGCATCTCTTGTTCAAGAGTTCCCTATCACTCTCTTTAATCACTACTCACAGTAACCTCAACTCCTG'
    p1_raw_df['sequence'] = ['{}{}'.format(s, il2mpseq) for s in p1_raw_df['sequence'].values]
    p1_raw_df = p1_raw_df.reset_index()
    del p1_raw_df['index']
    del p1_raw_df['count']
    return p1_raw_df

sense_cleanlist = [clean_read(path) for path in path_list]
sense_cleanlist[0].head()

,sequence
0,TCGATCTCCGCCCCCTCTTCGAAGATCAAAGGGTCGAGGGGACTTT...
1,TCGAATGAGTCACATCGAAGATCAAAGGGTCGACCCTTTGATCTTC...
2,TCGAGGGGACTTTCCTCGAGGGGGCGGGGTCGATTTGGCGCTCGAT...
3,TCGATTTCCAAGAAATCGAGGAAAGTCCCCTCGACCCTTTGATCTT...
4,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...


In [8]:
#extract shared sequences between high and low expressors
crosscat_sequences = pd.DataFrame(columns=['sequence'])
for i in range(3):
    common_tstlst = [sense_cleanlist[i], sense_cleanlist[3], sense_cleanlist[4], sense_cleanlist[5]]
    common = functools.reduce(lambda left,right: pd.merge(left,right,on='sequence'), common_tstlst)
    crosscat_sequences = pd.concat([crosscat_sequences, common])

#remove cross class sequences from each dataset
for i in range(len(sense_cleanlist)):
    sense_cleanlist[i] = sense_cleanlist[i][(~sense_cleanlist[i].sequence.isin(crosscat_sequences.sequence))]

for i in [0,1,3,4]:
    linked_common = functools.reduce(lambda left,right: pd.merge(left,right,on='sequence'), 
                                     [sense_cleanlist[i], sense_cleanlist[i+1]])
    sense_cleanlist[i] = sense_cleanlist[i][(~sense_cleanlist[i].sequence.isin(linked_common.sequence))]

In [9]:
for i in range(3):
    sense_cleanlist[i]['y'] = 0
for i in range(3, 6):
    sense_cleanlist[i]['y'] = 1
    
concat_df = pd.concat([df for df in sense_cleanlist]).reset_index(drop=True)
concat_df = concat_df.sample(frac=1).reset_index(drop=True)

c:\users\wolfgang\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\wolfgang\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
train, test = train_test_split(concat_df, test_size=0.1, random_state=42)
train_path = 'D:/Projects/iSynPro/iSynPro/HilbertCNN/train_val_npys/10count/train_df.csv'
test_path = 'D:/Projects/iSynPro/iSynPro/HilbertCNN/train_val_npys/10count/test_df.csv'
train.to_csv(train_path)
test.to_csv(test_path)